In [33]:
# This is the script to populate driver's Data
# {driver_id, time, curr_lat, curr_long, dest, load}

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', \
                    ' ip-172-31-0-105', 'ip-172-31-0-106'], \
                   port=9200)



def loadBoundaries(boundaries_file):
    boundaries = {}
    with open(boundaries_file, 'r') as f:
        reader = csv.reader(f, delimiter = ',')
        for row in reader:
            city = row[0]
            fence = row[1:] 
            boundaries.update({city:fence})
    f.close()
    return boundaries

def simulateTrip(id, c):
    step = 4
    q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        d = q['_source']['destination']
        c = q['_source']['location']
        return [c[0] + ((c[0] - d[0])/step), c[1] + ((c[1] - d[1])/step)]
    else:
        return [random.uniform(float(bnd[0]), float(bnd[2])), \
                random.uniform(float(bnd[1]),float(bnd[3]))]
        
        
def generateDriver(city):
    global last_uid 
    bnd = bound[city]
    last_uid += 1
    curr_lat = random.uniform(float(bnd[0]), float(bnd[2]))
    curr_long = random.uniform(float(bnd[1]),float(bnd[3]))
    
    driver_mapping ={ 
            'name': 'driver_{}'.format(last_uid),
            'id': random.randint(1, 20),
            'status': 'idle',
            'location': simulateTrip(),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
        }

    return(driver_mapping)

bound = loadBoundaries(boundaries_file)
for n in range(20):
    driver = generateDriver(city)
    u_json = json.dumps(driver).encode('utf-8')
    key = json.dumps(city).encode('utf-8')
    print('{}'.format(driver))
    producer.send(b'driver', key, u_json) 
    time.sleep(2)


IOError: [Errno 2] No such file or directory: 'boundaries.csv'

In [2]:

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', ' ip-172-31-0-105', 'ip-172-31-0-106'], port=9200)



In [14]:
q = {"fields" : ["destination"], "query":{ "id": 5} }

[40.78748930070537, -73.93026485004548]